# Imports 

In [1]:
from faker import Faker

In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
from pyspark.sql import functions as F

# Setup

In [5]:
spark = (
    SparkSession.builder.appName("pyspark-notebook")
    .master("spark://spark-master:7077")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-8487d920-f3c6-4043-b8cb-e62042c43638;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 292ms :: artifacts dl 11ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 

22/12/26 09:35:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", "minio")
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", "minio123")
spark._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "http://172.18.0.3:9000")

In [7]:
from delta.tables import *

# Create initial dataframe

In [8]:
fake = Faker()

In [9]:
data = [
    {"id": i, "name": fake.name(), "date": fake.date(), "address": fake.address()}
    for i in range(10)
]

In [10]:
df = spark.createDataFrame(data)

In [11]:
df.write.format("delta").save("s3a://lake/users")

22/12/26 09:35:22 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [12]:
df.select("*").orderBy("date").show()

+--------------------+----------+---+--------------------+
|             address|      date| id|                name|
+--------------------+----------+---+--------------------+
|748 Medina Ranch\...|1982-08-19|  0|          April Reed|
|5539 Lauren Track...|1984-05-15|  3|      Edwin Mcdonald|
|USNV Rogers\nFPO ...|1990-10-13|  5|         Paul Brooks|
|047 Lambert Court...|1990-12-29|  9|          Cody Brown|
|054 Allen Straven...|1991-10-11|  6|       Robert Barnes|
|51701 Kline Ports...|1998-08-12|  4|        Karen Foster|
|0013 Ramirez Vist...|2005-03-12|  1|         Damon Curry|
|Unit 4921 Box 019...|2006-11-14|  2|       Angela Howell|
|00807 Montgomery ...|2008-09-02|  7|Christopher Williams|
|1703 White Shoals...|2020-02-07|  8|    Kimberly Michael|
+--------------------+----------+---+--------------------+



In [13]:
df.createOrReplaceTempView("users")

# Add new data and change the schema 

In [14]:
data = [
    {"id": i, "name": fake.name(), "date": fake.date(), "address": fake.address(), "ip": fake.ipv4()}
    for i in range(10, 15)
]

In [15]:
df = spark.createDataFrame(data)

In [16]:
df.write.format("delta").option("mergeSchema", "true").mode("append").save("s3a://lake/users")

In [19]:
spark.sql("SELECT COUNT(*) FROM users").show()

+--------+
|count(1)|
+--------+
|      10|
+--------+



# Check schemas

In [20]:
dl = DeltaTable.forPath(spark, "s3a://lake/users")

In [21]:
dl.history().show(truncate=False, vertical=True)

-RECORD 0---------------------------------------------------------------------------
 version             | 1                                                            
 timestamp           | 2022-12-26 09:36:21                                          
 userId              | null                                                         
 userName            | null                                                         
 operation           | WRITE                                                        
 operationParameters | {mode -> Append, partitionBy -> []}                          
 job                 | null                                                         
 notebook            | null                                                         
 clusterId           | null                                                         
 readVersion         | 0                                                            
 isolationLevel      | Serializable                              

# Check data 

In [22]:
df = spark.read.format("delta").load("s3a://lake/users")

In [23]:
df.show()

+--------------------+----------+---+--------------------+---------------+
|             address|      date| id|                name|             ip|
+--------------------+----------+---+--------------------+---------------+
|9983 John Plaza\n...|1990-11-19| 12|   Latasha Dickerson|  83.58.118.251|
|438 Sheila Meadow...|2007-02-28| 13|           Juan Neal| 136.97.157.151|
|642 Jessica Keys ...|2017-07-08| 14|        Edward Smith| 219.111.47.235|
|65375 Tanya Light...|2010-11-24| 10|Christina Livingston|  181.81.92.107|
|0358 Diana Neck\n...|1996-09-08| 11|    Jordan Hernandez|160.124.112.146|
|USNV Rogers\nFPO ...|1990-10-13|  5|         Paul Brooks|           null|
|054 Allen Straven...|1991-10-11|  6|       Robert Barnes|           null|
|00807 Montgomery ...|2008-09-02|  7|Christopher Williams|           null|
|1703 White Shoals...|2020-02-07|  8|    Kimberly Michael|           null|
|047 Lambert Court...|1990-12-29|  9|          Cody Brown|           null|
|748 Medina Ranch\...|198

In [24]:
df_v0 = spark.read.format("delta").option("versionAsOf", 0).load("s3a://lake/users")

In [25]:
df_v1 = spark.read.format("delta").option("versionAsOf", 1).load("s3a://lake/users")

In [26]:
df_v0.count()

10

In [27]:
df_v1.count()

15

# Merge 

In [28]:
data = [
    {"id": 0, "name": fake.name()},
    {"id": 1000, "name": fake.name(), "date": fake.date(), "address": fake.address(), "ip": fake.ipv4()}

]

In [29]:
incoming_data = spark.createDataFrame(data)

In [30]:
incoming_data.show()

+----+-------------+--------------------+----------+--------------+
|  id|         name|             address|      date|            ip|
+----+-------------+--------------------+----------+--------------+
|   0|Shawn Bennett|                null|      null|          null|
|1000| George Brown|26456 Brown Field...|2010-12-03|132.187.12.127|
+----+-------------+--------------------+----------+--------------+



In [31]:
(
    dl
    .alias("users")
    .merge(incoming_data.alias("updates"),"users.id = updates.id")
    .whenMatchedUpdate(set = { "name" : "updates.name" } )
    .whenNotMatchedInsertAll()
    .execute()
)

In [32]:
dl.history().show()

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2022-12-26 09:40:33|  null|    null|    MERGE|{predicate -> (us...|null|    null|     null|          1|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      1|2022-12-26 09:36:21|  null|    null|    WRITE|{mode -> Append, ...|null|    null|     null|          0|  Serializable|         true|{numFiles -> 2, n...|        null|Apache-Spark/3.3....|
|      0|2022-1

In [33]:
spark.sql("SELECT * FROM users").show()

+--------------------+----------+---+--------------------+
|             address|      date| id|                name|
+--------------------+----------+---+--------------------+
|748 Medina Ranch\...|1982-08-19|  0|          April Reed|
|0013 Ramirez Vist...|2005-03-12|  1|         Damon Curry|
|Unit 4921 Box 019...|2006-11-14|  2|       Angela Howell|
|5539 Lauren Track...|1984-05-15|  3|      Edwin Mcdonald|
|51701 Kline Ports...|1998-08-12|  4|        Karen Foster|
|USNV Rogers\nFPO ...|1990-10-13|  5|         Paul Brooks|
|054 Allen Straven...|1991-10-11|  6|       Robert Barnes|
|00807 Montgomery ...|2008-09-02|  7|Christopher Williams|
|1703 White Shoals...|2020-02-07|  8|    Kimberly Michael|
|047 Lambert Court...|1990-12-29|  9|          Cody Brown|
+--------------------+----------+---+--------------------+



# Check the data again 

In [34]:
df = spark.read.format("delta").load("s3a://lake/users")

In [36]:
df.count()

16

In [37]:
df.show()

+--------------------+----------+----+--------------------+---------------+
|             address|      date|  id|                name|             ip|
+--------------------+----------+----+--------------------+---------------+
|748 Medina Ranch\...|1982-08-19|   0|       Shawn Bennett|           null|
|0013 Ramirez Vist...|2005-03-12|   1|         Damon Curry|           null|
|Unit 4921 Box 019...|2006-11-14|   2|       Angela Howell|           null|
|5539 Lauren Track...|1984-05-15|   3|      Edwin Mcdonald|           null|
|51701 Kline Ports...|1998-08-12|   4|        Karen Foster|           null|
|26456 Brown Field...|2010-12-03|1000|        George Brown| 132.187.12.127|
|9983 John Plaza\n...|1990-11-19|  12|   Latasha Dickerson|  83.58.118.251|
|438 Sheila Meadow...|2007-02-28|  13|           Juan Neal| 136.97.157.151|
|642 Jessica Keys ...|2017-07-08|  14|        Edward Smith| 219.111.47.235|
|65375 Tanya Light...|2010-11-24|  10|Christina Livingston|  181.81.92.107|
|0358 Diana 

# Vacuum 

In [ ]:
# dl.vacuum(0) 